In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e8/sample_submission.csv
/kaggle/input/playground-series-s5e8/train.csv
/kaggle/input/playground-series-s5e8/test.csv
/kaggle/input/bank-marketing/bank-full.csv


In [14]:
import numpy as np
import pandas as pd

# Load competition data
train = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')
test_id = test['id']

# Verify test_id length
print("Length of test_id:", len(test_id))
print("Test shape:", test.shape)
print("Train y unique values (raw):", train['y'].unique())
print("Train y value counts (raw):\n", train['y'].value_counts(dropna=False))

# Load original data
original = pd.read_csv('/kaggle/input/bank-marketing/bank-full.csv', sep=',')

# Map original y to 0/1
original['Target'] = original['Target'].map({'yes': 1, 'no': 0})
print("Original data y unique values:", original['Target'].unique())
print("Original data y NaN count:", original['Target'].isna().sum())
print("Original data y value counts:\n", original['Target'].value_counts(dropna=False))
print("Original shape:", original.shape)

# Add flags
original['is_original'] = 1
train['is_original'] = 0
test['is_original'] = 0

# Concat train + original, ensure no duplicates
full_train = pd.concat([train, original.drop('id', axis=1) if 'id' in original else original], ignore_index=True)
full_train.drop_duplicates(inplace=True)

# Combine with test
full_df = pd.concat([full_train, test], axis=0, ignore_index=True)
full_df['y'] = full_df['y'].fillna(-1)  # Dummy for test

# Verify shapes and y distribution
print("Full train shape:", full_train.shape)
print("Full train y value counts:\n", full_train['y'].value_counts(dropna=False))
print("Combined full_df shape:", full_df.shape)

Length of test_id: 250000
Test shape: (250000, 17)
Train y unique values (raw): [0 1]
Train y value counts (raw):
 y
0    659512
1     90488
Name: count, dtype: int64
Original data y unique values: [0 1]
Original data y NaN count: 0
Original data y value counts:
 Target
0    39922
1     5289
Name: count, dtype: int64
Original shape: (45211, 17)
Full train shape: (795211, 20)
Full train y value counts:
 y
0.0    659512
1.0     90488
NaN     45211
Name: count, dtype: int64
Combined full_df shape: (1045211, 20)


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [8]:
# Quick EDA
print(full_df.describe())
print(full_df.isnull().sum())  # No explicit nulls, but 'unknown' acts as missing
for col in ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']:
    print(f"{col} value counts:\n{full_df[col].value_counts(normalize=True)}\n")

# Correlation (numeric only)
num_cols = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
corr = full_df[num_cols + ['y']].corr()
print("Correlations with y:\n", corr['y'].sort_values(ascending=False))

                   id           age       balance           day      duration  \
count  1000000.000000  1.045211e+06  1.045211e+06  1.045211e+06  1.045211e+06   
mean    499999.500000  4.092824e+01  1.209322e+03  1.610349e+01  2.561007e+02   
std     288675.278933  1.011776e+01  2.823367e+03  8.256014e+00  2.716476e+02   
min          0.000000  1.800000e+01 -8.019000e+03  1.000000e+00  0.000000e+00   
25%     249999.750000  3.300000e+01  0.000000e+00  9.000000e+00  9.100000e+01   
50%     499999.500000  3.900000e+01  6.260000e+02  1.700000e+01  1.350000e+02   
75%     749999.250000  4.800000e+01  1.390000e+03  2.100000e+01  3.490000e+02   
max     999999.000000  9.500000e+01  1.021270e+05  3.100000e+01  4.918000e+03   

           campaign         pdays      previous             y   is_original  \
count  1.045211e+06  1.045211e+06  1.045211e+06  1.045211e+06  1.045211e+06   
mean   2.584262e+00  2.315029e+01  3.119734e-01 -1.958676e-01  4.325538e-02   
std    2.734182e+00  7.843260e+01

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# Replace 'unknown' with NaN
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
full_df[cat_cols] = full_df[cat_cols].replace('unknown', np.nan)

# Impute missing
cat_imputer = SimpleImputer(strategy='most_frequent')
full_df[cat_cols] = pd.DataFrame(cat_imputer.fit_transform(full_df[cat_cols]), columns=cat_cols)

# Binary cols to 0/1
binary_cols = ['default', 'housing', 'loan']
full_df[binary_cols] = full_df[binary_cols].replace({'yes': 1, 'no': 0})

# Handle pdays
full_df['pdays'] = full_df['pdays'].replace(-1, 999)
full_df['has_previous_contact'] = (full_df['pdays'] < 999).astype(int)

# Feature Engineering
full_df['age_bin'] = pd.cut(full_df['age'], bins=[0, 30, 40, 50, 60, 100], labels=[0,1,2,3,4])
full_df['balance_log'] = np.log1p(full_df['balance'].clip(lower=0))
full_df['balance_negative'] = (full_df['balance'] < 0).astype(int)
full_df['duration_bin'] = pd.cut(full_df['duration'], bins=[0, 100, 200, 300, 500, 1000, max(1001, full_df['duration'].max())], labels=[0,1,2,3,4,5])
full_df['duration_per_campaign'] = full_df['duration'] / (full_df['campaign'] + 1)
full_df['total_contacts'] = full_df['campaign'] + full_df['previous']
full_df['previous_success_rate'] = full_df['previous'] / (full_df['previous'] + full_df['campaign'] + 1)

# Handle month_num
month_map = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
full_df['month_num'] = full_df['month'].str.lower().map(month_map).fillna(0)
full_df['month_sin'] = np.sin(2 * np.pi * full_df['month_num'] / 12)
full_df['month_cos'] = np.cos(2 * np.pi * full_df['month_num'] / 12)

# Label encode categorical columns
for col in ['job', 'marital', 'education', 'contact', 'month', 'poutcome']:
    full_df[col] = full_df[col].fillna('missing')
    full_df[col] = LabelEncoder().fit_transform(full_df[col].astype(str))

full_df['job_edu_interact'] = full_df['job'] * full_df['education']

# Check for NaNs and dtypes
print("NaN counts in full_df:\n", full_df.isna().sum())
print("Dtypes in full_df:\n", full_df.dtypes)

# Split back, ensuring test_df matches test.csv
train_df = full_df[full_df['is_original'] == 1 | full_df['id'].isin(train['id'])].copy()
test_df = full_df[full_df['id'].isin(test['id'])].copy()  # Filter strictly by test IDs
test_df = test_df.drop('y', axis=1)

# Ensure y is binary and NaN-free
train_df['y'] = train_df['y'].map({1: 1, 0: 0, 'yes': 1, 'no': 0})
train_df = train_df[train_df['y'].isin([0, 1])]
print("Train y unique values:", train_df['y'].unique())
print("Train y NaN count:", train_df['y'].isna().sum())

# Verify shapes
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("Length of test_id:", len(test_id))

# Check for NaNs
X = train_df.drop('y', axis=1)
y = train_df['y']
print("NaN counts in X:\n", X.isna().sum())
print("NaN counts in test_df:\n", test_df.isna().sum())
print("Dtypes in X:\n", X.dtypes)

/tmp/ipykernel_36/165975301.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_df[binary_cols] = full_df[binary_cols].replace({'yes': 1, 'no': 0})


NaN counts in full_df:
 id                         45211
age                            0
job                            0
marital                        0
education                      0
default                        0
balance                        0
housing                        0
loan                           0
contact                        0
day                            0
month                          0
duration                       0
campaign                       0
pdays                          0
previous                       0
poutcome                       0
y                              0
is_original                    0
Target                   1000000
has_previous_contact           0
age_bin                        0
balance_log                    0
balance_negative               0
duration_bin                   3
duration_per_campaign          0
total_contacts                 0
previous_success_rate          0
month_num                      0
month_sin          

In [10]:
import optuna
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier, early_stopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [12]:
def objective_lgbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'class_weight': 'balanced',
        'random_state': 42,
        'objective': 'binary',
        'metric': 'auc'
    }
    model = LGBMClassifier(**params)
    skf = StratifiedKFold(n_splits=3)  # Quick for tuning
    aucs = []
    for train_idx, val_idx in skf.split(X, y):
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        preds = model.predict_proba(X.iloc[val_idx])[:, 1]
        aucs.append(roc_auc_score(y.iloc[val_idx], preds))
    return np.mean(aucs)

# # Run Optuna (uncomment to tune; takes time, ~30 min)
# study = optuna.create_study(direction='maximize')
# study.optimize(objective_lgbm, n_trials=50)
# best_params_lgbm = study.best_params
# print(best_params_lgbm)

# For now, use sample tuned params (replace with your Optuna results)
best_params_lgbm = {'n_estimators': 800, 'learning_rate': 0.03, 'max_depth': 7, 'num_leaves': 50, 'subsample': 0.8, 'colsample_bytree': 0.8}

In [13]:
import optuna
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier, early_stopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Define categorical features
cat_features = ['job', 'marital', 'education', 'contact', 'month', 'poutcome', 'age_bin', 'duration_bin']
print("Categorical features for CatBoost:", cat_features)
print("Dtypes of cat_features in X:\n", X[cat_features].dtypes)

# Sample tuned params
best_params_lgbm = {'n_estimators': 800, 'learning_rate': 0.03, 'max_depth': 7, 'num_leaves': 50, 'subsample': 0.8, 'colsample_bytree': 0.8}

models = {
    'lgbm': LGBMClassifier(
        **best_params_lgbm,
        class_weight='balanced',
        random_state=42,
        objective='binary',
        metric='auc',
        verbose=-1
    ),
    'xgb': XGBClassifier(
        n_estimators=800,
        learning_rate=0.03,
        max_depth=7,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=len(y[y==0])/len(y[y==1]),
        random_state=42,
        enable_categorical=True
    ),
    'cat': CatBoostClassifier(
        iterations=800,
        learning_rate=0.03,
        depth=7,
        subsample=0.8,
        random_state=42,
        auto_class_weights='Balanced',
        verbose=0
    )
}

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
test_preds = np.zeros((len(test_df), len(models)))
auc_scores = {name: [] for name in models}

for name, model in models.items():
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        if name == 'cat':
            X_train_cat = X_train.copy()
            X_val_cat = X_val.copy()
            test_df_cat = test_df.copy()
            for col in ['age_bin', 'duration_bin']:
                X_train_cat[col] = X_train_cat[col].astype(str)
                X_val_cat[col] = X_val_cat[col].astype(str)
                test_df_cat[col] = test_df_cat[col].astype(str)
            model.fit(X_train_cat, y_train, eval_set=(X_val_cat, y_val), early_stopping_rounds=50, cat_features=cat_features)
            test_pred = model.predict_proba(test_df_cat)[:, 1]
        elif name == 'lgbm':
            model.fit(
                X_train, y_train, 
                eval_set=[(X_val, y_val)], 
                eval_metric='auc',
                callbacks=[early_stopping(stopping_rounds=50, verbose=False)]
            )
            test_pred = model.predict_proba(test_df)[:, 1]
        else:  # xgb
            model.fit(
                X_train, y_train, 
                eval_set=[(X_val, y_val)], 
                eval_metric='auc',
                early_stopping_rounds=50,
                verbose=False
            )
            test_pred = model.predict_proba(test_df)[:, 1]
        
        val_pred = model.predict_proba(X_val_cat if name == 'cat' else X_val)[:, 1]
        auc = roc_auc_score(y_val, val_pred)
        auc_scores[name].append(auc)
        print(f"{name} Fold {fold+1} AUC: {auc:.4f}")
        print(f"Shape of test_pred for {name} Fold {fold+1}:", test_pred.shape)
        
        test_preds[:, list(models).index(name)] += test_pred / skf.n_splits
    
    print(f"Average {name} AUC: {np.mean(auc_scores[name]):.4f}")

# Verify final test_preds shape
print("Shape of test_preds:", test_preds.shape)
print("Length of final_test_preds:", len(final_test_preds))

# Ensemble
weights = [0.4, 0.3, 0.3]
final_test_preds = np.average(test_preds, axis=1, weights=weights)

Categorical features for CatBoost: ['job', 'marital', 'education', 'contact', 'month', 'poutcome', 'age_bin', 'duration_bin']
Dtypes of cat_features in X:
 job                int64
marital            int64
education          int64
contact            int64
month              int64
poutcome           int64
age_bin         category
duration_bin    category
dtype: object


ZeroDivisionError: division by zero

In [ ]:
# Verify target variable
print("Unique values in y:", y.unique())
if len(y.unique()) > 2:
    print("Warning: y contains more than two classes. Fixing to binary...")
    y = y.map({'yes': 1, 'no': 0, 1: 1, 0: 0})  # Ensure binary mapping
    print("After mapping, unique values in y:", y.unique())